In [1]:
import os
os.chdir("../")
import h5py
import numpy as np
import geopandas as gpd
from osgeo import gdal
import rasterio
from rasterio.transform import from_origin
from blackmarble.utils import bm_raster_i

In [2]:
h5file = os.getcwd() + "/data/VNP46A2.A2023001.h32v10.001.2023012014343.h5"

In [3]:
hdflayer = gdal.Open(h5file, gdal.GA_ReadOnly)

subhdflayer = hdflayer.GetSubDatasets()[0][0]
rlayer = gdal.Open(subhdflayer, gdal.GA_ReadOnly)

outputRaster = "test.tif"

HorizontalTileNumber = int(rlayer.GetMetadata_Dict()["HorizontalTileNumber"])
VerticalTileNumber = int(rlayer.GetMetadata_Dict()["VerticalTileNumber"])

WestBoundCoord = (10 * HorizontalTileNumber) - 180
NorthBoundCoord = 90 - (10 * VerticalTileNumber)
EastBoundCoord = WestBoundCoord + 10
SouthBoundCoord = NorthBoundCoord - 10

EPSG = "-a_srs EPSG:4326"  #WGS84

translateOptionText = EPSG + " -a_ullr " + str(WestBoundCoord) + " " + str(
    NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(
        SouthBoundCoord)

translateoptions = gdal.TranslateOptions(
    gdal.ParseCommandLine(translateOptionText))
gdal.Translate(outputRaster, rlayer, options=translateoptions)

/Users/czhang/miniforge3/envs/ox/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x16830bf60> >

In [4]:
raster_bp = rasterio.open("test_bp.tif")
raster_bp.meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 2400,
 'height': 2400,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.004166666666666667, 0.0, 140.0001220703125,
        0.0, -0.004166666666666667, -10.0)}

In [8]:
import rioxarray
from shapely.ops import Point
r = rioxarray.open_rasterio("test.tif")
r = r.squeeze().drop(["spatial_ref", "band"])
r.name = "ntl"
rdf = r.to_dataframe().reset_index()
rdf["geometry"] = [Point(x,y) for x, y in zip(rdf["y"], rdf["x"])]

In [9]:
rdf["ntl"] = rdf["ntl"].replace(65535, np.nan)
rdf_ntl = gpd.GeoDataFrame(rdf[rdf.ntl.isna() == False].reset_index(drop=True))


In [10]:
rdf_ntl.head(50).explore()

In [5]:
np.unique(raster_bp.read())

array([    0,     1,     2,     3,     4,     5,     6, 65535],
      dtype=uint16)

In [6]:
raster_official = rasterio.open("test.tif")
raster_official.meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 2400,
 'height': 2400,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.004166666666666667, 0.0, 140.0,
        0.0, -0.004166666666666667, -10.0)}

In [7]:
bm_tiles_path = os.getcwd() + "/data/BlackMarbleTiles/"
bm_tiles = gpd.read_file(bm_tiles_path)
bounds = bm_tiles[bm_tiles["TileID"] == "h32v10"].geometry.bounds
xMin = float(bounds.minx.iloc[0])
yMin = float(bounds.miny.iloc[0])
xMax = float(bounds.maxx.iloc[0])
yMax = float(bounds.maxy.iloc[0])

In [8]:
h5f = h5py.File(h5file)
data = out = h5f["HDFEOS"]["GRIDS"]["VNP_Grid_DNB"]["Data Fields"]["DNB_BRDF-Corrected_NTL"]

In [9]:
output_path = "test_bp.tif"
nRows = out.shape[0]
nCols = out.shape[1]
res = nRows
nodata_val = 65535
myCrs = 4326

data = out

# Define the pixel size and number of rows and columns
pixel_size = 1  # Size of each pixel in the output raster
rows, cols = data.shape

# Define the spatial extent (bounding box) of the raster
left, bottom, right, top = xMin, yMin, xMax, yMax

psize_x = (xMax - xMin) / cols
psize_y = (yMax - yMin) / rows

# Create the raster file
with rasterio.open(
        output_path,
        "w",
        driver="GTiff",
        height=rows,
        width=cols,
        count=1,
        dtype=data.dtype,
        crs="EPSG:4326",
        transform=from_origin(left, top, psize_x, psize_y),
) as dst:
    dst.write(data, 1)